<a href="https://colab.research.google.com/github/DiogoSilvaa/Stock-Prediction/blob/master/stockscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
#Import libraries
import pandas_datareader as web
import numpy as np
from sklearn.preprocessing import scale
import pandas as pd
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [250]:
def preprocessing(df):
  #Normalise data 
  for c in df.columns:
      df[c] = df[c].pct_change()
      df[c] = scale(df[c])
      df.dropna(inplace=True)
  
  df.dropna(inplace=True)

  return np.array(df)

In [251]:
#Get data date range
today = datetime.now()
delta = timedelta(days = 400)
past = today - delta

today = today.date()
past = past.date()

In [252]:
#Import stock data
stocks = ['QQQ','DIA','SPY']
main_df = pd.DataFrame()
for index in stocks:
  df = web.DataReader(index, data_source = 'yahoo', start=past, end=today) #Import data from Yahoo Finance (starting from 01-01-2000 to 01-01-2021)
  
  df[f"150SMA_{index}"] = df.Close.rolling(150).mean()  # Add a column to include the last 150 days simple moving average
  df[f"dSMA_{index}"] = df['Close'] - df[f"150SMA_{index}"] # Add a column to include distance of closing price to 150 SMA

  df.rename(columns={"Close": f"close_{index}", "Volume": f"volume_{index}"}, inplace=True)
  df = df[[f"close_{index}",f"volume_{index}", f"dSMA_{index}"]]
  
  if len(main_df) == 0:
    main_df = df
  else:
    main_df = main_df.join(df)

main_df.dropna(inplace=True)

main_df = main_df[-109:]

In [253]:
#Data processing and neural network feeding 
last100 = preprocessing(main_df)
last100 = np.reshape(last100,(1, 100, last100.shape[1]))

model_nq = load_model('/content/models/QQQ.h5')
model_sp = load_model('/content/models/SPY.h5')
model_dw = load_model('/content/models/DIA.h5')

#Model prediction
nq = (model_nq.predict(last100))
sp = (model_sp.predict(last100))
dw = (model_dw.predict(last100))

if nq[0][0]>nq[0][1]:
  nq = 0
else:
  nq = 1

if sp[0][0]>sp[0][1]:
  sp = 0
else:
  sp = 1

if dw[0][0]>dw[0][1]:
  dw = 0
else:
  dw = 1

pred = [["QQQ",nq],["SPY",sp],["DIA",dw]]

NQ: [[0.1525434 0.8474566]]
SP: [[0.8218324  0.17816752]]
Dow: [[0.67185724 0.32814276]]
[['QQQ', 1], ['SPY', 0], ['DIA', 0]]
